In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# 파일 경로를 지정하세요
file_path = 'merged_filtered_road.csv'  # 실제 파일 경로로 변경하세요

# 데이터 불러오기 함수 정의
def load_data(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='euc-kr')
        except UnicodeDecodeError as e:
            print("파일을 읽는 데 실패했습니다.")
            print(e)
            exit()
    return df

# 데이터 필터링 및 필요한 열 선택 함수 정의
def filter_data(df):
    required_columns = ['경도', '위도', 'ARI']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"누락된 열이 있습니다: {missing_columns}")
        exit()
    
    result_df = df[required_columns].dropna(subset=['경도', '위도', 'ARI'])
    print(f"경도, 위도, ARI 값이 있는 데이터 수: {len(result_df)}")
    return result_df

# ARI 값에 따라 색상 및 단계 계산 함수 정의
def get_color_and_stage(ari, min_ari, max_ari):
    normalized_value = (ari - min_ari) / (max_ari - min_ari)
    colors = ["#00FF00", "#FFFF00", "#FFA500", "#FF0000"]
    stages = [1, 2, 3, 4]
    
    for i, threshold in enumerate([0.25, 0.5, 0.75, 1.0]):
        if normalized_value <= threshold:
            return colors[i], stages[i]
    
    return colors[-1], stages[-1]  # 최대값 초과 시 마지막 단계 반환

# 지도 생성 및 표시 함수 정의
def create_map(df):
    center_lat = df['위도'].mean()
    center_lon = df['경도'].mean()
    mymap = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    max_ari = df['ARI'].max()
    min_ari = df['ARI'].min()
    
    for idx, row in df.iterrows():
        color, stage = get_color_and_stage(row['ARI'], min_ari, max_ari)
        
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"ARI: {row['ARI']}, 단계: {stage}"
        ).add_to(mymap)
    
    # 범례 추가
    legend_html = '''
    {% macro html(this, kwargs) %}
    <div style="position: fixed; bottom: 50px; right: 50px; width: 120px; height: 120px; 
                border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                ">&nbsp; <b>ARI 단계</b> <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FF0000"></i> 4단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FFA500"></i> 3단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FFFF00"></i> 2단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#00FF00"></i> 1단계 <br>
    </div>
    {% endmacro %}
    '''
    
    macro = MacroElement()
    macro._template = Template(legend_html)
    mymap.get_root().add_child(macro)
    
    return mymap

# 메인 함수
def main():
    df = load_data(file_path)
    result_df = filter_data(df)
    
    if result_df.empty:
        print("필터링된 데이터가 없습니다. 프로그램을 종료합니다.")
        return
    
    mymap = create_map(result_df)
    return mymap

# 실행
mymap = main()
mymap